In [ ]:
import os, sys
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
%cd dm_crawling
!ls -al

In [ ]:
!curl -s https://raw.githubusercontent.com/teddylee777/machine-learning/master/99-Misc/01-Colab/mecab-colab.sh | bash

In [ ]:
!apt-get update -qq
!apt-get install fonts-nanum* -qq

import matplotlib.pyplot as plt
import matplotlib.font_manager as fm

In [ ]:
# 시스템에 설치된 폰트 확인 및 원하는 폰트 전체 경로 불러오기
system_fonts = fm.findSystemFonts()
nanum_fonts = [font for font in system_fonts if 'Nanum' in font]
print(f"Nanum font path: {nanum_fonts}")

# 원하는 폰트 경로를 matplotlib 기본 설정에 적용
path = '/usr/share/fonts/truetype/nanum/NanumGothicCodingBold.ttf'
font_name = fm.FontProperties(fname=path, size=10).get_name()
plt.rc('font', family=font_name)

# 주피터 노트북 전체에 적용
!echo "import matplotlib.pyplot as plt" >> ~/.ipython/profile_default/startup/startup.py
!echo "plt.rc('font', family='$font_name')" >> ~/.ipython/profile_default/startup/startup.py

# matplotlib 폰트 캐시 삭제
!rm -rf ~/.cache/matplotlib

In [ ]:
from google.colab import files
uploaded = files.upload()

In [ ]:
%matplotlib inline

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import requests
import json

In [ ]:
import re

# 텍스트 데이터에서 한글만 남기고 모두 제거하는 함수
def text_cleaning(text):
    hangul = re.compile('[^ ㄱ-ㅣ가-힣]+')
    result = hangul.sub('', str(text))
    return result

In [ ]:
from konlpy.tag import Okt

# 형태소(PoS) 추출
def get_pos(x):
    tagger = Okt()
    pos = tagger.pos(x)
    pos = ['{}/{}'.format(word,tag) for word, tag in pos]
    return pos

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer

from sklearn.model_selection import train_test_split

from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

from sklearn.metrics import confusion_matrix

from sklearn.cluster import KMeans

import matplotlib.pyplot as plt

In [ ]:
# 삭제할 불용어
stopwords = ['대/Modifier', '나/Josa', '기/Modifier', '이고/Josa', '고/Josa', '인/Josa', '인데/Josa', '가/Josa', '가/Verb', '가까스로/Noun', '가령/Noun', '가서/Verb', '각/Noun', '각각/Noun', '각자/Noun', '각종/Noun', '각하/Noun', '간/Noun', '갖고말하자면/Verb', '같다/Adjective', '같다면/Adjective', '같은/Adjective', '같이/Adverb', '같이/Josa', '개/Noun', '거나/Noun', '거니와/Eomi', '거리/Noun', '거바/Noun', '거의/Noun', '걸/Noun', '겁/Noun', '것/Noun', '것과/Noun', '게다가/Noun', '게우다/Verb', '겨우/Noun', '견지/Noun', '결과/Noun', '결국/Adverb', '결론/Noun', '겸사겸사/Noun', '고려/Noun', '고로/Noun', '곤/Noun', '곧/Noun', '곳/Noun', '공동/Noun', '과/Josa', '과/Noun', '과연/Noun', '관/Noun', '관계/Noun', '관계없이/Adverb', '관련/Noun', '구/Noun', '구체/Noun', '구토/Noun', '그/Determiner', '그/Noun', '그냥/Noun', '그때/Noun', '그래/Adjective', '그래도/Adverb', '그래서/Adverb', '그러나/Conjunction', '그러니/Adjective', '그러니까/Adverb', '그러면/Adverb', '그러므로/Conjunction', '그러한/Adjective', '그런/Adjective', '그런/Modifier', '그런데/Conjunction', '그럼/Adjective', '그럼에도/Adjective', '그렇게/Adverb', '그렇지/Adjective', '그렇지만/Conjunction', '그리고/Conjunction', '그리하여/Conjunction', '그저/Adverb', '그치지/Verb', '근거/Noun', '기대/Noun', '기점/Noun', '기준/Noun', '기타/Noun', '김/Noun', '까닭/Noun', '까악/Adverb', '까지/Josa', '까지도/Josa', '꽈당/Adverb', '끙끙/Noun', '끼익/Noun', '나/Noun', '나머지/Noun', '남/Noun', '남짓/Noun', '낫다/Noun', '내/Noun', '낼/Noun', '너/Noun', '너희/Noun', '네/Noun', '넷/Noun', '년/Noun', '년도/Noun', '논/Noun', '놀라다/Verb', '누가/Noun', '누구/Noun', '는/Josa', '다/Adverb', '다/Josa', '다른/Noun', '다를가/Adjective', '다만/Noun', '다섯/Noun', '다소/Noun', '다수/Noun', '다시/Noun', '다시말하면/Verb', '다음/Noun', '단지/Noun', '달려/Verb', '답/Noun', '당신/Noun', '당장/Noun', '대로/Noun', '대하/Noun', '대해/Noun', '댕그/Noun', '더/Noun', '더구나/Noun', '더군다나/Noun', '더라도/Noun', '더불어/Verb', '더욱더/Noun', '더욱이/Noun', '도/Josa', '도/Noun', '도달/Noun', '도착/Noun', '동시/Noun', '동안/Noun', '됏/Noun', '되는/Verb', '되다/Verb', '되면/Verb', '되어/Verb', '된/Verb', '된다/Verb', '될것이다/Verb', '두번째/Noun', '둘/Noun', '둥둥/Noun', '뒤/Noun', '뒤따라/Verb', '든/Josa', '든/Verb', '든지/Josa', '들/Suffix', '들/Verb', '들면/Verb', '들자면/Verb', '듯/Noun', '등/Noun', '등등/Noun', '딩동/Noun', '따라/Verb', '따라서/Noun', '따르는/Verb', '따름/Noun', '따위/Noun', '따지지/Verb', '딱/Adverb', '때/Noun', '때문/Noun', '또/Noun', '또한/Noun', '뚝뚝/Noun', '라/Josa', '라도/Josa', '라면/Noun', '령/Noun', '로/Josa', '로/Noun', '로부터/Noun', '로써/Noun', '륙/Noun', '를/Josa', '를/Noun', '리하/Noun', '마음대로/Noun', '마자/Noun', '마저/Noun', '마저도/Josa', '마치/Noun', '막론/Noun', '만/Josa', '만/Noun', '만약/Noun', '만으로/Josa', '만은/Josa', '만이/Josa', '만이/Noun', '만일/Noun', '만큼/Josa', '만큼/Noun', '만하/Noun', '많은/Adjective', '말/Noun', '말하자면/Verb', '망정/Noun', '매/Noun', '매번/Noun', '메/Noun', '면/Josa', '몇/Noun', '모/Noun', '모두/Noun', '모른다/Verb', '몰라도/Verb', '몰랏다/Noun', '못/VerbPrefix', '무렵/Noun', '무릎/Noun', '무슨/Noun', '무엇/Noun', '물론/Adverb', '뭐/Noun', '미치다/Adjective', '및/Noun', '바/Noun', '바꾸어말하면/Verb', '바꾸어말하자면/Verb', '바꾸어서/Verb', '바꿔/Verb', '바로/Noun', '밖에/Josa', '반대/Noun', '반드시/Noun', '방면/Noun', '버금/Noun', '번째/Suffix', '보는데서/Verb', '보다/Verb', '보드득/Noun', '보면/Verb', '보아/Noun', '본대/Noun', '봐/Verb', '봐라/Verb', '부류/Noun', '부터/Noun', '불구/Noun', '불문/Noun', '붕붕/Noun', '비/Noun', '비걱거리/Noun', '비교/Noun', '비길수/Verb', '비로소/Noun', '비록/Noun', '비슷하다/Adjective', '비추어/Verb', '뿐/Noun', '삐걱/Noun', '사/Noun', '사람/Noun', '삼/Noun', '상대/Noun', '생각/Noun', '서/Josa', '서술/Noun', '설령/Noun', '설마/Noun', '설사/Noun', '셋/Noun', '소생/Noun', '소인/Noun', '솨/Noun', '수/Noun', '쉿/Noun', '습니까/Eomi', '습니다/Eomi', '시각/Noun', '시간/Noun', '시오/Noun', '시작/Noun', '시초/Noun', '시키다/Verb', '실로/Noun', '심지어/Noun', '쓰/Verb', '쓰고/Verb', '쓰여/Verb', '아/Exclamation', '아니/Adjective', '아니나/Adjective', '아니다/Adjective', '아니라/Adjective', '아니라면/Adjective', '아니면/Adjective', '아니었다면/Adjective', '아래/Noun', '아무/Modifier', '아무/Noun', '아야/Noun', '아울러/Verb', '아이/Noun', '아이고/Exclamation', '아이구/Exclamation', '아이야/Noun', '아이쿠/Exclamation', '아하/Noun', '아홉/Noun', '안/Noun', '안다/Verb', '안된다/Adjective', '않고/Verb', '않기/Verb', '않는다면/Verb', '않다/Verb', '않다면/Verb', '않도록/Verb', '않으면/Verb', '알/Noun', '알겠는가/Verb', '알다/Noun', '알았어/Verb', '앗/Noun', '앞/Noun', '야/Exclamation', '약간/Noun', '양자/Noun', '어/Eomi', '어/Suffix', '어기/Noun', '어느/Adverb', '어느/Modifier', '어디/Noun', '어때/Adjective', '어떠한/Adjective', '어떤/Adjective', '어떤/Modifier', '어떤것/Adjective', '어떻게/Adjective', '어떻해/Adjective', '어이/Noun', '어째서/Noun', '어쨋/Noun', '어쩔/Modifier', '어찌/Noun', '어찌됏/Noun', '어찌하여/Noun', '언제/Noun', '언젠가/Adverb', '얼마/Noun', '얼마간/Noun', '얼마나/Noun', '얼마만큼/Noun', '없고/Adjective', '없다/Adjective', '엉엉/Noun', '에/Josa', '에게/Josa', '에도/Josa', '에서/Josa', '에야/Josa', '여/Josa', '여/Modifier', '여/Noun', '여기/Noun', '여덟/Noun', '여러분/Noun', '여보/Noun', '여부/Noun', '여섯/Noun', '여전히/Adjective', '연/Modifier', '연관/Noun', '영/Noun', '영차/Noun', '옆/Noun', '예/Noun', '예컨대/Adverb', '오/Modifier', '오/Noun', '오로지/Adverb', '오르다/Noun', '오직/Noun', '오호/Exclamation', '오히려/Noun', '와/Josa', '와/Verb', '와같이/Verb', '와르르/Noun', '와아/Exclamation', '왜/Noun', '왜냐하면/Adverb', '외/Noun', '요/Josa', '요/Noun', '요만/Modifier', '요컨대/Noun', '우/Noun', '우르르/Noun', '우리/Noun', '우선/Noun', '운운/Verb', '월/Noun', '위/Noun', '위해/Noun', '윗/Noun', '윙윙/Noun', '육/Noun', '으로/Josa', '으로서/Josa', '으로써/Josa', '은/Josa', '은/Noun', '을/Josa', '응/Noun', '응당/Noun', '의/Josa', '의/Noun', '의거/Noun', '의지/Noun', '의치/Noun', '의해/Adjective', '의해서/Adjective', '이/Determiner', '이/Josa', '이/Noun', '이다/Josa', '이래/Adjective', '이러/Verb', '이러하다/Adjective', '이러한/Adjective', '이런/Adjective', '이럴/Adjective', '이렇게/Adverb', '이렇구나/Adjective', '이로/Noun', '이르기까지/Verb', '이르다/Verb', '이번/Noun', '이상/Noun', '이서/Noun', '이어/Verb', '이어서/Adverb', '이었다/Verb', '이외/Noun', '이용/Noun', '이유/Noun', '이지만/Josa', '이천구/Noun', '이천육/Noun', '이천칠/Noun', '이천팔/Noun', '인/Noun', '인젠/Noun', '인하여/Adjective', '일/Modifier', '일/Noun', '일것이다/Verb', '일곱/Noun', '일단/Noun', '일반/Noun', '일지/Noun', '임/Noun', '입/Noun', '입니다/Adjective', '입어/Verb', '입장/Noun', '잇따라/Verb', '있다/Adjective', '자/Noun', '자기/Noun', '자마자/Noun', '자신/Noun', '잠깐/Noun', '잠시/Noun', '저/Determiner', '저/Noun', '저기/Noun', '저쪽/Noun', '저희/Noun', '적/Noun', '적/Suffix', '전부/Noun', '전자/Noun', '전후/Noun', '점/Noun', '정도/Noun', '제/Noun', '제각기/Noun', '제외/Noun', '젠/Noun', '조금/Noun', '조차/Noun', '조차도/Josa', '졸졸/Noun', '좀/Noun', '종합/Noun', '좋다/Adjective', '좋아/Adjective', '좍좍/Noun', '주룩주룩/Noun', '주저/Noun', '줄/Noun', '중/Noun', '즈음/Noun', '즉/Noun', '즉시/Noun', '지/Josa', '지경/Noun', '지든지/Verb', '지만/Josa', '지말고/Verb', '진짜/Noun', '집/Noun', '쪽/Noun', '차/Noun', '차라리/Noun', '참/Verb', '참나/Noun', '첫/Noun', '쳇/Exclamation', '총/Modifier', '칠/Noun', '켠으로는/Verb', '콸콸/Noun', '쾅쾅/Adverb', '쿵/Noun', '큼/Verb', '타다/Verb', '타인/Noun', '탕탕/Adverb', '토/Noun', '통/Noun', '툭/Noun', '퉤/Noun', '틀림없다/Adjective', '틈타/Verb', '팍/Noun', '팔/Noun', '퍽/Noun', '펄렁/Noun', '편이/Noun', '하/Exclamation', '하게/Verb', '하게하다/Verb', '하겠는가/Verb', '하고/Josa', '하고/Verb', '하고있었다/Verb', '하구나/Verb', '하기/Verb', '하기는한데/Verb', '하기만/Verb', '하기보다는/Verb', '하기에/Verb', '하나/Noun', '하느니/Verb', '하는/Verb', '하는것도/Verb', '하는것만/Verb', '하는것이/Verb', '하다/Verb', '하더라도/Verb', '하도다/Verb', '하도록/Verb', '하든/Noun', '하든지/Verb', '하려고하다/Verb', '하마터면/Adverb', '하면/Verb', '하면서/Verb', '하물며/Noun', '하여/Verb', '하여금/Adverb', '하여야/Verb', '하였다/Verb', '하자/Noun', '하자면/Verb', '하지/Verb', '하지마/Verb', '하지마라/Verb', '하지만/Conjunction', '하하/Noun', '한/Determiner', '한/Josa', '한/Modifier', '한/Verb', '한다/Verb', '한다면/Verb', '한데/Eomi', '한마디/Noun', '한적/Noun', '한하다/Verb', '할/Verb', '할것도/Verb', '할수록/Verb', '할수있다/Verb', '할수있어/Verb', '할줄/Verb', '할지라도/Verb', '할지언정/Verb', '함/Noun', '함께/Adverb', '항목/Noun', '해/Noun', '해도/Noun', '해봐요/Verb', '해서/Verb', '해서는/Verb', '해야/Verb', '했어요/Verb', '향/Noun', '허/Verb', '허걱/Noun', '허허/Adverb', '헉/Adverb', '헉헉/Adverb', '헐떡헐떡/Adverb', '형식/Noun', '혹시/Noun', '혹은/Adverb', '혼자/Noun', '후/Noun', '훨씬/Adverb', '휘익/Noun', '휴/Exclamation', '흐흐/Noun', '흥/Noun', '힘/Noun']

In [ ]:
df = pd.read_excel(list(uploaded.keys())[0], engine = 'openpyxl')
df['ko_text'] = df['content'].apply(lambda x: text_cleaning(x))
del df['content']
# print(df.head(3))

# 형태소를 벡터 형태의 X 데이터로 변환
index_vectorizer = CountVectorizer(tokenizer = lambda x: get_pos(x), stop_words=stopwords, token_pattern=None)
X = index_vectorizer.fit_transform(df['ko_text'].tolist())
# print(X.shape)

# X 데이터에 대해 TF-IDF로 학습 데이터셋으로 변환
tfidf_vectorizer = TfidfTransformer()
X = tfidf_vectorizer.fit_transform(X)
# print(X.shape)
# print(X[0])

# 1:1 비율로 랜덤 샘플링 수행
positive_random_idx = df[df['y']==1].sample(50, random_state=30, replace=True).index.tolist()
negative_random_idx = df[df['y']==0].sample(50, random_state=30, replace=True).index.tolist()

# 랜덤 데이터로 데이터셋 나누기 (7:3 비율)
random_idx = positive_random_idx + negative_random_idx
sample_X = X[random_idx, :]
y = df['y'][random_idx]
x_train, x_test, y_train, y_test = train_test_split(sample_X,y,test_size=0.30)
print(x_train.shape)
print(x_test.shape)

# 로지스틱 회귀 모델 학습
lr = LogisticRegression(random_state=0)
lr.fit(x_train, y_train)
y_pred = lr.predict(x_test)
y_pred_probability = lr.predict_proba(x_test)[:,1]

# 모델 성능 평가
print("accuracy: %.2f" % accuracy_score(y_test, y_pred))
print("Precision : %.3f" % precision_score(y_test, y_pred))
print("Recall : %.3f" % recall_score(y_test, y_pred))
print("F1 : %.3f" % f1_score(y_test, y_pred))

# Confusion Matrix 출력
confmat = confusion_matrix(y_true=y_test, y_pred=y_pred)
print(confmat)

# 학습한 회귀 모델 계수 출력
plt.rcParams['figure.figsize'] = [10, 8]
plt.bar(range(len(lr.coef_[0])), lr.coef_[0])
print(sorted(((value, index) for index, value in enumerate(lr.coef_[0])), reverse=True)[:5])
print(sorted(((value, index) for index, value in enumerate(lr.coef_[0])), reverse=True)[-5:])

# 회귀 모델의 계수를 높은 순으로 정렬
coef_pos_index = sorted(((value,index) for index, value in enumerate(lr.coef_[0])), reverse=True)

# 회귀 모델의 계수를 index_vectorizer에 맵핑한 뒤 형태소 출력
invert_index_vectorizer = {v: k for k, v in index_vectorizer.vocabulary_.items()}

# 계수가 높은 순으로 피처-형태소 맵핑
print(str(invert_index_vectorizer)[:100]+'..')

# 상위 20개 긍정 형태소
for coef in coef_pos_index[:20]:
    print(invert_index_vectorizer[coef[1]], coef[0])

# 하위 20개 긍정 형태소 (부정 형태소)
for coef in coef_pos_index[-20:]:
    print(invert_index_vectorizer[coef[1]], coef[0])

In [ ]:
kmeans = KMeans(n_clusters=2)
kmeans.fit(X)

labels = kmeans.labels_
df['cluster'] = labels

# 클러스터링 결과 출력
for i in range(2):
    print(f"Cluster {i}:")
    print(df[df['cluster'] == i])
    print()

# 클러스터링 결과 시각화
plt.hist(df['cluster'])
plt.title('Histogram of Cluster Labels')
plt.xlabel('Cluster')
plt.ylabel('Number of Reviews')
plt.show()

In [ ]:
def remove_pos_info(word):
    # 품사 정보 제거 ('/Noun', '/Verb', 등 )
    return re.sub('/\w+', '', word)

In [ ]:
from wordcloud import WordCloud
import matplotlib.pyplot as plt

# 계수가 높은 순으로 피처-형태소 맵핑
coef_pos_index = sorted(((value, index) for index, value in enumerate(lr.coef_[0])), reverse=True)

# 상위 50개 긍정 형태소 출력
positive_words = {remove_pos_info(invert_index_vectorizer[coef[1]]): coef[0] for coef in coef_pos_index[:50]}
positive_wordcloud = WordCloud(font_path=path, background_color='white').generate_from_frequencies(positive_words)

plt.figure(figsize=(10, 10))
plt.imshow(positive_wordcloud, interpolation="bilinear")
plt.axis('off')
plt.title('Positive Review Words')
plt.show()